107b3772fc6d44d29c888c282949c2b4

In [4]:
from pystac_client import Client
from pystac_client import Client
import planetary_computer

# Obtain your API key and use it here
api_key = '107b3772fc6d44d29c888c282949c2b4'


In [5]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime="2010-01-01/2022-12-31",
    bbox=[-120.0, 35.0, -119.0, 36.0],
    query={"eo:cloud_cover": {"lt": 10}},
)

items = list(search.get_items())
print(f"Found {len(items)} items")

# Sign the assets
for item in items:
    for asset_key in item.assets:
        item.assets[asset_key].href = planetary_computer.sign(item.assets[asset_key].href)


/home/jake/.conda/envs/firelab/lib/python3.9/site-packages/pystac_client/item_search.py:834: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Found 2414 items


In [7]:
import stackstac
import xarray as xr

# Convert items to a stackstac stack
# Assuming that each item's asset has been signed, and using the correct band names
stack = stackstac.stack(
    [item.to_dict() for item in items],
    assets=["B4", "B8"],  # Red and NIR bands for NDVI calculation
    chunksize=4096,
    resolution=10
)

# Compute NDVI
ndvi = (stack.sel(asset='B8') - stack.sel(asset='B4')) / (stack.sel(asset='B8') + stack.sel(asset='B4'))
# add a progress bar
import tqdm
ndvi = ndvi.compute(progressbar_type='tqdm')
# ndvi = ndvi.compute()  # Depending on the size, this might take time

TypeError: stack() got an unexpected keyword argument 'chunk_size'

In [ ]:
ndvi.to_netcdf("ndvi_data_2010_2022.nc")
